In [4]:
import pandas as pd
import datetime 

airports_codes = ['TLV', 'LHR', 'MAD', 'JFK', 'CDG', 'FCO', 'ATH', 'AMS', 'FRA']
dataset = pd.read_csv("raw_data.csv")
                         
# Extract airport codes from 'Origin' and 'Destination' columns, filter bad values
dataset['OriginCode'] = dataset['Origin'].str.extract('\((.*?)\)')
dataset['DestinationCode'] = dataset['Destination'].str.extract('\((.*?)\)')
dataset = dataset[dataset['OriginCode'].isin(airports_codes) & dataset['DestinationCode'].isin(airports_codes)]

# extract the aircraft code
dataset = dataset.assign(Aircraft_Code = dataset['Aircraft'].str.split('(').str[0])
dataset['Aircraft_Code'] = dataset['Aircraft_Code'].str.replace(" ", "")

#extract the airline code, filter bad values
dataset['Code_letters'] = dataset['Code'].str.extract('^([A-Z]+)')
airlines_codes = ['DLH', 'KLM', 'AEE', 'BAW', 'IBE', 'DAL', 'AFR', 'ITY', 'ELY', 'DL', 'KL', 'AE', 'BA', 'IB', 'DA', 'AF', 'IT', 'EL']
dataset = dataset[dataset['Code_letters'].isin(airlines_codes)]

# Convert the time strings to datetime objects
dataset['Scheduled Departure'] = pd.to_datetime(dataset['Scheduled Departure'], format='%H:%M')
dataset['Actual Departure'] = pd.to_datetime(dataset['Actual Departure'], format='%H:%M', errors='coerce')

# Add the offset column
dataset['offset'] = dataset['Actual Departure'] - dataset['Scheduled Departure']

# devide to 3 bins, to map dual hours sections of the day in which the flight is scheduled
bin_start_1 = pd.to_datetime('00:00', format='%H:%M')
bin_end_1 = pd.to_datetime('00:59', format='%H:%M')
bins = [bin_start_1,bin_end_1]
labels = [0]
for i in range(1,24):
    if i<10:
        str1='0'+str(i)+':00'
    else:
        str1=str(i)+':00' 
    bin1 = pd.to_datetime(str1, format='%H:%M')
    bins.append(bin1)
    labels.append(i)

bins.sort()
dataset['time_bin'] = pd.cut(dataset['Scheduled Departure'], bins=bins, labels=labels, include_lowest=True)

#Convert the time columns back to string and format them, filter bad values
dataset['Scheduled Departure'] = dataset['Scheduled Departure'].dt.strftime('%H:%M')
dataset['Actual Departure'] = dataset['Actual Departure'].dt.strftime('%H:%M')
dataset['offset1'] = pd.to_timedelta(dataset['offset'])
dataset['offset_minutes'] = dataset['offset1'].dt.total_seconds() / 60
dataset = dataset.drop('offset1', axis=1)
offset_hours = dataset['offset'].dt.total_seconds() / 3600
offset_hours1 = dataset['offset'].dt.total_seconds() / 3600
dataset['offset']  = offset_hours.fillna(0).apply(lambda x: '{:02d}:{:02d}'.format(int(x), int((x % 1) * 60)))
dataset[dataset['offset']>"20:00"]
dataset=dataset.drop(dataset[dataset['offset']>"20:00"].index)

#Convert aircraft to maximum seats possible
seats_dict = {'B788': 330,
              'B789': 350,
              'B772': 380,
              'A35K': 440,
              'B77W': 440,
              'A388': 853,
              'B744': 660,
              'A332': 440,
              'A359': 440,
              'A333': 440,
              'A339': 440,
              'B764': 440,
              'B763': 440,
              'A321': 220,
              'B739': 150,
              'A320': 150,
              'A21N': 150,
              'A20N': 150,
              'A319': 150,
              'A318': 150,
              'B738': 150,
              'B737': 150,
              'E190': 100,
              'E75L': 70,
              'E295': 90,
              'BCS3': 80,
              'E170': 70,
              'CRJX': 70,
              'B78X': 70,
              'J328': 50,
              'B752': 220}
dataset['Aircraft_Code'] = dataset['Aircraft_Code'].map(seats_dict)
dataset = dataset.rename(columns={'Aircraft_Code': 'Aircraft_Seats'})

airport_codes_dict = {'TLV' : 1,
                      'LHR' : 2,
                      'MAD' : 3,
                      'JFK' : 4,
                      'CDG' : 5,
                      'FCO' : 6,
                      'ATH' : 7,
                      'AMS' : 8,
                      'FRA' : 9}
#Convert origin and destination to code
#unique_values = dataset['OriginCode'].unique()
#mapping_OriginCode = {value: index+1 for index, value in enumerate(unique_values)}
dataset['OriginCode'] = dataset['OriginCode'].map(airport_codes_dict)
    
#unique_values = dataset['DestinationCode'].unique()
#mapping_DestinationCode = {value: index+1 for index, value in enumerate(unique_values)}
dataset['DestinationCode'] = dataset['DestinationCode'].map(airport_codes_dict)

airlines_codes_dict = {'DLH' : 1, 
                       'KLM' : 2, 
                       'AEE' : 3, 
                       'BAW' : 4, 
                       'IBE' : 5, 
                       'DAL' : 6, 
                       'AFR' : 7, 
                       'ITY' : 8, 
                       'ELY' : 9, 
                       'DL' : 1, 
                       'KL' : 2, 
                       'AE' : 3, 
                       'BA' : 4, 
                       'IB' : 5, 
                       'DA' : 6, 
                       'AF' : 7, 
                       'IT' : 8, 
                       'EL' : 9}
#unique_values = dataset['Code_letters'].unique()
#mapping_Code_letters = {value: index+1 for index, value in enumerate(unique_values)}
dataset['Code_letters'] = dataset['Code_letters'].map(airlines_codes_dict)

#Output the relevant, relatively clean dataset
dataset = dataset.drop(dataset.columns[0:9], axis=1)
dataset = dataset.drop(dataset.columns[4], axis=1)
dataset = dataset.reset_index(drop=True)
dataset.to_csv("clean_data.csv")
